我们在这里得到视觉中心并保存，我们这里的视觉中心用伪标签的样本得到

In [1]:
# 我们在给样本打上伪标签之后，就可以在视觉空间中找出视觉特征每个类别的中心，然后在视觉空间中进行KNN搜索类别。
# 使用训练集的样本以及伪标签，经过模型得到视觉特征后，可以得到视觉特征中心。
# 然后使用测试集的样本，经过模型得到视觉特征，然后和视觉特征中心进行KNN搜索，得到预测的类别。

In [2]:
import sys
sys.path.append('/mnt/workspace')

In [3]:
from py_file.M_attri import Att
from py_file.Get_Data import DATA
from py_file.data_set import MyDataSet
from torch.utils.data import DataLoader,Dataset,random_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import numpy as np
import matplotlib.pyplot as plt
import os
import math

In [4]:
# 定义训练的设备
if torch.cuda.is_available():
    device = torch.device('cuda:0') # 只有一张显卡的话，'cuda'和'cuda:0'是一样的
else:
    device = torch.device('cpu')

print(f'使用的设备为：{device}')

使用的设备为：cuda:0


In [5]:
trans = transforms.Resize(224)  # ResNet模型适合的图片大小为224x244
# 输入的张量需要带着批次维度和通道维度

# 准备数据以及模型

In [7]:
# 读取模型以及语义属性
import pickle
model = torch.load('../model_saved_pseudo/train_all.pth')
# model = model.to(device)
# model.eval()

model_fea = torch.nn.Sequential(*list(model.children())[:-2])  # 由于加了sigmoid层，所以这里是[:-2]
model_fea = model_fea.to(device)
model_fea.eval()

'''
attri = Att()
attri.compute_mul_defect_att()

single_defect_att = attri.single_defect_att
two_defect_att = attri.two_defect_att
three_defect_att = attri.three_defect_att
four_defect_att = attri.four_defect_att
mul_defect_att = attri.mul_defect_att
total_defect_att = attri.total_defect_att
'''

with open('../updated_semantic_all/updated_single_dict.pkl', 'rb') as file:
    single_defect_att = pickle.load(file)
with open('../updated_semantic_all/updated_mul_dict.pkl', 'rb') as file:
    mul_defect_att = pickle.load(file)
with open('../updated_semantic_all/updated_total_dict.pkl', 'rb') as file:
    total_defect_att = pickle.load(file)

from collections import Counter
# 加载二、三、四缺陷的字典
two_defect_att = {}
three_defect_att = {}
four_defect_att = {}
for label in mul_defect_att.keys():
    count = Counter(label)
    if count['+'] == 1:
        two_defect_att[label] = mul_defect_att[label]
    if count['+'] == 2:
        three_defect_att[label] = mul_defect_att[label]
    if count['+'] == 3:
        four_defect_att[label] = mul_defect_att[label]

In [8]:
train_data_path = '/mnt/workspace/DATA/train_WM.npz'
train_data = np.load(train_data_path)

pseudo_two_data_path = '../data_fake_label/two_fake_label_WM.npz' 
pseudo_two_data = np.load(pseudo_two_data_path)

pseudo_three_data_path = '../data_fake_label/three_fake_label_WM.npz' 
pseudo_three_data = np.load(pseudo_three_data_path)

pseudo_four_data_path = '../data_fake_label/four_fake_label_WM.npz' 
pseudo_four_data = np.load(pseudo_four_data_path)

att_dimen = len(single_defect_att['Center'])
print('att_dimen:', att_dimen)

att_dimen: 20


In [9]:
train_label = train_data['label_name']
pseudo_two_label = pseudo_two_data['label_name']
pseudo_three_label = pseudo_three_data['label_name']
pseudo_four_label = pseudo_four_data['label_name']

In [10]:
train_wm = train_data['denoise_wm']
train_wm_tensor = trans(torch.reshape(torch.tensor(train_wm, dtype=torch.float32),(len(train_wm),1,52,52)))
print(train_wm_tensor.shape)

pseudo_two_wm = pseudo_two_data['denoise_wm']
pseudo_two_wm_tensor = trans(torch.reshape(torch.tensor(pseudo_two_wm, dtype=torch.float32),(len(pseudo_two_wm),1,52,52)))
print(pseudo_two_wm_tensor.shape)

pseudo_three_wm = pseudo_three_data['denoise_wm']
pseudo_three_wm_tensor = trans(torch.reshape(torch.tensor(pseudo_three_wm, dtype=torch.float32),(len(pseudo_three_wm),1,52,52)))
print(pseudo_three_wm_tensor.shape)

pseudo_four_wm = pseudo_four_data['denoise_wm']
pseudo_four_wm_tensor = trans(torch.reshape(torch.tensor(pseudo_four_wm, dtype=torch.float32),(len(pseudo_four_wm),1,52,52)))
print(pseudo_four_wm_tensor.shape)

torch.Size([25910, 1, 224, 224])
torch.Size([2593, 1, 224, 224])
torch.Size([4130, 1, 224, 224])
torch.Size([1629, 1, 224, 224])


In [11]:
train_label_oh = train_data['label_one_hot']
# 通过one_hot标签给数据分为单缺陷、双缺陷、三缺陷以及四缺陷

train_single_wm = []  # 先定义列表，然后转换为tensor
train_single_label = []

train_two_wm = []
train_two_label = []

train_three_wm = []
train_three_label = []

train_four_wm = []
train_four_label = []
for i in range(len(train_label_oh)):
    if train_label_oh[i].sum() <= 1:
        train_single_wm.append(np.array(train_wm_tensor[i]))
        train_single_label.append(train_label[i])

    elif train_label_oh[i].sum() == 2:
        train_two_wm.append(np.array(train_wm_tensor[i]))
        train_two_label.append(train_label[i])

    elif train_label_oh[i].sum() == 3:
        train_three_wm.append(np.array(train_wm_tensor[i]))
        train_three_label.append(train_label[i])

    elif train_label_oh[i].sum() == 4:
        train_four_wm.append(np.array(train_wm_tensor[i]))
        train_four_label.append(train_label[i])


del train_data,train_wm_tensor

# 定义需要的函数

In [12]:
def compute_visual_centers(X, labels, category):
    """
    计算每个类别的视觉中心（均值特征向量）。
    :param X: 特征向量，形状 (num_samples, feature_dim)
    :param labels: 每个样本的标签，形状 (num_samples,)
    :param category: 缺陷的类别种类
    :return: 每个类别的视觉中心（均值向量），是一个字典
    """
    visual_centers = {}
    for c in category:
        visual_centers[c] = np.mean(X[labels == c], axis=0)
    return visual_centers

# 获取视觉中心并保存

In [13]:
import pickle

## 单故障

In [14]:
category = list(single_defect_att.keys())
single_wm_tensor = torch.tensor(np.array(train_single_wm), dtype=torch.float32)
single_dataset = MyDataSet(single_wm_tensor,train_single_label)
single_loader = DataLoader(single_dataset, batch_size=32, shuffle=False)
del single_wm_tensor

In [15]:
x = []
y = np.array(train_single_label)
with torch.no_grad():
    for imgs,labels in single_loader:  
        imgs = imgs.to(device)
        outputs = model_fea(imgs)

        for out in outputs:
            x.append(out.flatten().cpu())  # 输出展开,out的维度大小变为512，没有flatten的话维度大小就是(512,1,1)

x = np.array(x)
print(x.shape, y.shape)

(5610, 512) (5610,)


In [16]:
del single_dataset, single_loader

In [17]:
single_vc = compute_visual_centers(x, y ,category)
print(single_vc.keys())

dict_keys(['Center', 'Donut', 'Edge_loc', 'Loc', 'Edge_ring', 'Scratch', 'Random', 'Nearfull', 'Normal'])


In [18]:
with open('single_vc.pkl', 'wb') as file:
    pickle.dump(single_vc, file)  # pickle 模块的dump函数，将数据写入文件，pickle可以写入任何类型的数据

## 二故障

### 获取视觉中心

In [19]:
category = list(two_defect_att.keys())
two_dataset = MyDataSet(pseudo_two_wm_tensor,pseudo_two_label)
two_loader = DataLoader(two_dataset, batch_size=32, shuffle=False)
del pseudo_two_wm_tensor

In [20]:
x = []
y = np.array(pseudo_two_label)
with torch.no_grad():
    for imgs,labels in two_loader:  
        imgs = imgs.to(device)
        outputs = model_fea(imgs)

        for out in outputs:
            x.append(out.flatten().cpu())  # 输出展开,out的维度大小变为512，没有flatten的话维度大小就是(512,1,1)

x = np.array(x)
print(x.shape, y.shape)

(2593, 512) (2593,)


In [21]:
del two_dataset, two_loader

In [22]:
two_vc = compute_visual_centers(x, y ,category)
print(two_vc.keys())

dict_keys(['C+EL', 'C+ER', 'C+L', 'C+S', 'D+EL', 'D+ER', 'D+L', 'D+S', 'EL+L', 'EL+S', 'ER+L', 'ER+S', 'L+S'])


In [23]:
with open('two_vc.pkl', 'wb') as file:
    pickle.dump(two_vc, file)  # pickle 模块的dump函数，将数据写入文件，pickle可以写入任何类型的数据

## 三故障

### 获取视觉中心

In [24]:
category = list(three_defect_att.keys())
three_dataset = MyDataSet(pseudo_three_wm_tensor,pseudo_three_label)
three_loader = DataLoader(three_dataset, batch_size=32, shuffle=False)
del pseudo_three_wm_tensor

In [25]:
x = []
y = np.array(pseudo_three_label)
with torch.no_grad():
    for imgs,labels in three_loader:  
        imgs = imgs.to(device)
        outputs = model_fea(imgs)

        for out in outputs:
            x.append(out.flatten().cpu())  # 输出展开,out的维度大小变为512，没有flatten的话维度大小就是(512,1,1)

x = np.array(x)
print(x.shape, y.shape)

(4130, 512) (4130,)


In [26]:
del three_dataset, three_loader

In [27]:
three_vc = compute_visual_centers(x, y ,category)
print(three_vc.keys())

dict_keys(['C+EL+L', 'C+EL+S', 'C+ER+L', 'C+ER+S', 'C+L+S', 'D+EL+L', 'D+EL+S', 'D+ER+L', 'D+ER+S', 'D+L+S', 'EL+L+S', 'ER+L+S'])


In [28]:
with open('three_vc.pkl', 'wb') as file:
    pickle.dump(three_vc, file)  # pickle 模块的dump函数，将数据写入文件，pickle可以写入任何类型的数据

## 四故障

### 获取视觉中心

In [29]:
category = list(four_defect_att.keys())
four_dataset = MyDataSet(pseudo_four_wm_tensor,pseudo_four_label)
four_loader = DataLoader(four_dataset, batch_size=32, shuffle=False)
del pseudo_four_wm_tensor

In [30]:
x = []
y = np.array(pseudo_four_label)
with torch.no_grad():
    for imgs,labels in four_loader:  
        imgs = imgs.to(device)
        outputs = model_fea(imgs)

        for out in outputs:
            x.append(out.flatten().cpu())  # 输出展开,out的维度大小变为512，没有flatten的话维度大小就是(512,1,1)

x = np.array(x)
print(x.shape, y.shape)

(1629, 512) (1629,)


In [31]:
del four_dataset, four_loader

In [32]:
four_vc = compute_visual_centers(x, y ,category)
print(four_vc.keys())

dict_keys(['C+EL+L+S', 'C+ER+L+S', 'D+EL+L+S', 'D+ER+L+S'])


In [33]:
with open('four_vc.pkl', 'wb') as file:
    pickle.dump(four_vc, file)  # pickle 模块的dump函数，将数据写入文件，pickle可以写入任何类型的数据

## 所有故障

### 获取视觉中心

In [34]:
total_vc = {**single_vc, **two_vc, **three_vc, **four_vc}
print(total_vc.keys())

dict_keys(['Center', 'Donut', 'Edge_loc', 'Loc', 'Edge_ring', 'Scratch', 'Random', 'Nearfull', 'Normal', 'C+EL', 'C+ER', 'C+L', 'C+S', 'D+EL', 'D+ER', 'D+L', 'D+S', 'EL+L', 'EL+S', 'ER+L', 'ER+S', 'L+S', 'C+EL+L', 'C+EL+S', 'C+ER+L', 'C+ER+S', 'C+L+S', 'D+EL+L', 'D+EL+S', 'D+ER+L', 'D+ER+S', 'D+L+S', 'EL+L+S', 'ER+L+S', 'C+EL+L+S', 'C+ER+L+S', 'D+EL+L+S', 'D+ER+L+S'])


In [35]:
with open('total_vc.pkl', 'wb') as file:
    pickle.dump(total_vc, file)  # pickle 模块的dump函数，将数据写入文件，pickle可以写入任何类型的数据